In [ ]:
#pip install paho-mqtt

In [34]:
## Publishing to MQTT + subscribing and printing the message and metadata

import paho.mqtt.client as mqtt
import time

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, message):
    print("message received " ,str(message.payload.decode("utf-8")))
    print("message topic=",message.topic)
    print("message qos=",message.qos)
    print("message retain flag=",message.retain)

publish_key = "pub-c-defc7b94-6e9a-42fd-b25a-883c2e605a89"  #"<your publish key>"
subscribe_key = "sub-c-5a7d9a20-e5a9-11eb-bc98-fa4b4b21bdc4"    #"<your subscribe key>"
client_id = "E4_1"     #"<your unique client identifier>"

print("creating new instance")

client = mqtt.Client(client_id=publish_key + "/" + subscribe_key + "/" + client_id)
client.on_message = on_message
print("connecting to broker")
client.connect("mqtt.pndsn.com", 1883, 60)

client.loop_start() #start the loop

print("Subscribing to topic","E4")
client.subscribe("E4")
print("Publishing message to topic","E4")
client.publish("E4", "Hello World")

time.sleep(4) # wait
client.loop_stop() #stop the loop

creating new instance
connecting to broker
Subscribing to topic E4
Publishing message to topic E4
message received  Hello World
message topic= E4
message qos= 0
message retain flag= 0


In [49]:
## Publishing to MQTT + subscribing and printing the message and metadata from live E4 data


import socket
import time
import pylsl
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import threading
%matplotlib qt
#%matplotlib inline

# SELECT DATA TO STREAM
acc = False      # 3-axis acceleration
bvp = False      # Blood Volume Pulse
gsr = False      # Galvanic Skin Response (Electrodermal Activity)
tmp = True       # Temperature
ibi = False      # Interbeat Interval and Heartbeat
bat = False      # Device Battery
tag = False      # Tag taken from the device (by pressing the button)

serverAddress = '127.0.0.1'
serverPort = 28000
bufferSize = 4096

deviceID = 'CDCB11' # 'A03003'

def connect():
    global s
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(3)

    print("Connecting to server")
    s.connect((serverAddress, serverPort))
    print("Connected to server\n")

    print("Devices available:")
    s.send("device_list\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Connecting to device")
    s.send(("device_connect " + deviceID + "\r\n").encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Pausing data receiving")
    s.send("pause ON\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
connect()

time.sleep(1)

def suscribe_to_data():
    if acc:
        print("Suscribing to ACC")
        s.send(("device_subscribe " + 'acc' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bvp:
        print("Suscribing to BVP")
        s.send(("device_subscribe " + 'bvp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if gsr:
        print("Suscribing to GSR")
        s.send(("device_subscribe " + 'gsr' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tmp:
        print("Suscribing to Temp")
        s.send(("device_subscribe " + 'tmp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if ibi:
        print("Suscribing to IBI")
        s.send(("device_subscribe " + 'ibi' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bat:
        print("Suscribing to Batt")
        s.send(("device_subscribe " + 'bat' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tag:
        print("Suscribing to Tag")
        s.send(("device_subscribe " + 'tag' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))

    print("Resuming data receiving")
    s.send("pause OFF\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
suscribe_to_data()

time.sleep(1)

def reconnect():
    print("Reconnecting...")
    connect()
    suscribe_to_data()
    #stream()

def disconnect():
    s.send("device_disconnect\r\n".encode())


""" response = s.recv(bufferSize).decode("utf-8")
    #print(response)
    if "connection lost to device" in response:
        print(response.decode("utf-8"))
        reconnect() """



import paho.mqtt.client as mqtt
import time

#messages = []
temp_timestamp = []
temp_data = []

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, message):
    print("message received " ,str(message.payload.decode("utf-8")))
    print("message topic=",message.topic)
    print("message qos=",message.qos)
    print("message retain flag=",message.retain)
    #messages.append(message.payload.decode("utf-8"))
    new_message = message.payload.decode("utf-8")
    samples = new_message.split("\n")
    for i in range(len(samples)-1):
        stream_type = samples[i].split()[0]
        if stream_type == "E4_Temperature":
            temp_new_timestamp = float(samples[i].split()[1].replace(',','.'))
            temp_new_data = float(samples[i].split()[2].replace(',','.'))
            temp_timestamp.append(temp_new_timestamp)
            temp_data.append(temp_new_data)
    

publish_key = "pub-c-defc7b94-6e9a-42fd-b25a-883c2e605a89"  #"<your publish key>"
subscribe_key = "sub-c-5a7d9a20-e5a9-11eb-bc98-fa4b4b21bdc4"    #"<your subscribe key>"
client_id = "E4_1"     #"<your unique client identifier>"

print("creating new instance")

client = mqtt.Client(client_id=publish_key + "/" + subscribe_key + "/" + client_id)
client.on_message = on_message
print("connecting to broker")
client.connect("mqtt.pndsn.com", 1883, 60)

client.loop_start() #start the loop

print("Subscribing to topic","E4")
client.subscribe("E4_TB")
print("Publishing message to topic","E4")
while True:
    response = s.recv(bufferSize).decode("utf-8")
    client.publish("E4_TB", response)
    #time.sleep(40) # wait


client.loop_stop() #stop the loop
disconnect()


Connecting to server
Connected to server

Devices available:
R device_list 1 | CDCB11 Empatica_E4 

Connecting to device
R device_connect OK

Pausing data receiving
R pause ON

Suscribing to Temp
R device_subscribe tmp OK

Resuming data receiving
R pause OFF

creating new instance
connecting to broker
Subscribing to topic E4
Publishing message to topic E4
message received  E4_Temperature 1626394667.00535 31.61
E4_Temperature 1626394667.25536 31.61
E4_Temperature 1626394667.50538 31.61
E4_Temperature 1626394667.75539 31.61
E4_Temperature 1626394668.0054 31.61
E4_Temperature 1626394668.25541 31.61
E4_Temperature 1626394668.50542 31.61
E4_Temperature 1626394668.75543 31.61

message topic= E4_TB
message qos= 0
message retain flag= 0
message received  E4_Temperature 1626394669.00545 31.61

message topic= E4_TB
message qos= 0
message retain flag= 0
message received  E4_Temperature 1626394669.25546 31.61
E4_Temperature 1626394669.50547 31.61
E4_Temperature 1626394669.75548 31.61

message topi

KeyboardInterrupt: 

In [52]:
disconnect()

In [50]:
temp_data

[31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.61,
 31.59,
 31.59,
 31.59,
 31.59]

In [51]:
temp_timestamp

[1626394667.00535,
 1626394667.25536,
 1626394667.50538,
 1626394667.75539,
 1626394668.0054,
 1626394668.25541,
 1626394668.50542,
 1626394668.75543,
 1626394669.00545,
 1626394669.25546,
 1626394669.50547,
 1626394669.75548,
 1626394670.00549,
 1626394670.2555,
 1626394670.50551,
 1626394670.75553,
 1626394671.00554,
 1626394671.25555,
 1626394671.50556,
 1626394671.75557,
 1626394672.00558,
 1626394672.2556,
 1626394672.50561,
 1626394672.75562]

In [33]:
messages

['E4_Temperature 1626393934.97146 31.69\r\nE4_Temperature 1626393935.22147 31.69\r\n',
 'E4_Temperature 1626393935.47149 31.69\r\nE4_Temperature 1626393935.7215 31.69\r\nE4_Temperature 1626393935.97151 31.69\r\nE4_Temperature 1626393936.22152 31.69\r\nE4_Temperature 1626393936.47153 31.69\r\nE4_Temperature 1626393936.72154 31.69\r\n',
 'E4_Temperature 1626393936.97155 31.69\r\n',
 'E4_Temperature 1626393937.22157 31.69\r\nE4_Temperature 1626393937.47158 31.69\r\nE4_Temperature 1626393937.72159 31.69\r\n',
 'E4_Temperature 1626393937.9716 31.71\r\nE4_Temperature 1626393938.22161 31.71\r\n',
 'E4_Temperature 1626393938.47162 31.71\r\nE4_Temperature 1626393938.72164 31.71\r\n',
 'E4_Temperature 1626393938.97165 31.69\r\nE4_Temperature 1626393939.22166 31.69\r\n',
 'E4_Temperature 1626393939.47167 31.69\r\nE4_Temperature 1626393939.72168 31.69\r\nE4_Temperature 1626393939.97169 31.69\r\nE4_Temperature 1626393940.2217 31.69\r\nE4_Temperature 1626393940.47172 31.69\r\nE4_Temperature 16263939

In [34]:
for i in messages:
    print (i)

E4_Temperature 1626393934.97146 31.69
E4_Temperature 1626393935.22147 31.69

E4_Temperature 1626393935.47149 31.69
E4_Temperature 1626393935.7215 31.69
E4_Temperature 1626393935.97151 31.69
E4_Temperature 1626393936.22152 31.69
E4_Temperature 1626393936.47153 31.69
E4_Temperature 1626393936.72154 31.69

E4_Temperature 1626393936.97155 31.69

E4_Temperature 1626393937.22157 31.69
E4_Temperature 1626393937.47158 31.69
E4_Temperature 1626393937.72159 31.69

E4_Temperature 1626393937.9716 31.71
E4_Temperature 1626393938.22161 31.71

E4_Temperature 1626393938.47162 31.71
E4_Temperature 1626393938.72164 31.71

E4_Temperature 1626393938.97165 31.69
E4_Temperature 1626393939.22166 31.69

E4_Temperature 1626393939.47167 31.69
E4_Temperature 1626393939.72168 31.69
E4_Temperature 1626393939.97169 31.69
E4_Temperature 1626393940.2217 31.69
E4_Temperature 1626393940.47172 31.69
E4_Temperature 1626393940.72173 31.69

E4_Temperature 1626393940.97174 31.67

E4_Temperature 1626393941.22175 31.67
E4_Tem

In [16]:
samples_test = messages.split("\n")

AttributeError: 'list' object has no attribute 'split'

In [17]:
"test" + "test"

'testtest'

In [9]:
for i in messages:
        stream_type = messages[i].split()[0]
        print(stream_type)

TypeError: list indices must be integers or slices, not str

In [7]:
for i in messages:
    print(i)

E4_Temperature 1626392588.90914 30.95
E4_Temperature 1626392589.15916 30.95
E4_Temperature 1626392589.40917 30.95
E4_Temperature 1626392589.65918 30.95
E4_Temperature 1626392589.90919 30.95
E4_Temperature 1626392590.1592 30.95
E4_Temperature 1626392590.40921 30.95
E4_Temperature 1626392590.65922 30.95

E4_Temperature 1626392590.90924 30.97
E4_Temperature 1626392591.15925 30.97

E4_Temperature 1626392591.40926 30.97
E4_Temperature 1626392591.65927 30.97
E4_Temperature 1626392591.90928 30.95
E4_Temperature 1626392592.15929 30.95

E4_Temperature 1626392592.40931 30.95
E4_Temperature 1626392592.65932 30.95

E4_Temperature 1626392592.90933 30.95

E4_Temperature 1626392593.15934 30.95
E4_Temperature 1626392593.40935 30.95
E4_Temperature 1626392593.65936 30.95
E4_Temperature 1626392593.90937 30.95
E4_Temperature 1626392594.15939 30.95
E4_Temperature 1626392594.4094 30.95
E4_Temperature 1626392594.65941 30.95

E4_Temperature 1626392594.90942 30.95
E4_Temperature 1626392595.15943 30.95

E4_Temp